In [ ]:
! pip install kaggle

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import torch
import torchaudio
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.dataset import TensorDataset
import torchvision
from torchvision import transforms

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

from torch.utils.data import Dataset, DataLoader
import os

from IPython.display import Audio
from tqdm import tqdm

upload kaggle token

sign in > settings > API > create new token

In [ ]:
from google.colab import files
files.upload()

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download andradaolteanu/gtzan-dataset-music-genre-classification

Dataset URL: https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification
License(s): other
 98% 1.19G/1.21G [00:13<00:00, 138MB/s]
100% 1.21G/1.21G [00:13<00:00, 93.9MB/s]


In [ ]:
! unzip gtzan-dataset-music-genre-classification

Archive:  gtzan-dataset-music-genre-classification.zip
  inflating: Data/features_30_sec.csv  
  inflating: Data/features_3_sec.csv  
  inflating: Data/genres_original/blues/blues.00000.wav  
  inflating: Data/genres_original/blues/blues.00001.wav  
  inflating: Data/genres_original/blues/blues.00002.wav  
  inflating: Data/genres_original/blues/blues.00003.wav  
  inflating: Data/genres_original/blues/blues.00004.wav  
  inflating: Data/genres_original/blues/blues.00005.wav  
  inflating: Data/genres_original/blues/blues.00006.wav  
  inflating: Data/genres_original/blues/blues.00007.wav  
  inflating: Data/genres_original/blues/blues.00008.wav  
  inflating: Data/genres_original/blues/blues.00009.wav  
  inflating: Data/genres_original/blues/blues.00010.wav  
  inflating: Data/genres_original/blues/blues.00011.wav  
  inflating: Data/genres_original/blues/blues.00012.wav  
  inflating: Data/genres_original/blues/blues.00013.wav  
  inflating: Data/genres_original/blues/blues.00014.wa

In [ ]:
df = pd.read_csv('./Data/features_3_sec.csv')
#df = pd.read_csv('./Data/features_30_sec.csv')
df.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.0.wav,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues
1,blues.00000.1.wav,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues
2,blues.00000.2.wav,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues
3,blues.00000.3.wav,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues
4,blues.00000.4.wav,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues


In [ ]:
df['label'].value_counts()

,count
label,
blues,1000
jazz,1000
metal,1000
pop,1000
reggae,1000
disco,999
classical,998
hiphop,998
rock,998


In [ ]:
label = list(df['label'].unique())
# convert categorical data into numerical values
df['label'] = df['label'].apply(lambda x: label.index(x))
df['label'].value_counts()

,count
label,
0,1000
5,1000
6,1000
7,1000
8,1000
3,999
1,998
4,998
9,998


In [ ]:
train_data, test_data = train_test_split(df, train_size = 0.8, random_state = 1)
t = pd.DataFrame(test_data)
t['label'].value_counts()

,count
label,
6,218
2,215
4,212
1,206
0,204
8,199
7,196
5,194
9,178


In [ ]:
# normalize the data
scaler = StandardScaler()
scaled_train_features = scaler.fit_transform(train_data.drop(['filename', 'length', 'label'], axis=1))
scaled_test_features = scaler.transform(test_data.drop(['filename', 'length', 'label'], axis=1))

In [ ]:
# create pytorch datasets and data loaders
train_labels = torch.tensor(train_data['label'].values, dtype=int)
test_labels = torch.tensor(test_data['label'].values, dtype=int)


In [ ]:
train_dataset = TensorDataset(torch.tensor(scaled_train_features, dtype=torch.float32), train_labels)
test_dataset = TensorDataset(torch.tensor(scaled_test_features, dtype=torch.float32), test_labels)


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
model = nn.Sequential(
    nn.BatchNorm1d(scaled_train_features.shape[1]),
    nn.Linear(scaled_train_features.shape[1], 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, 16),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(16, 10),
)

In [ ]:
print(model)

Sequential(
  (0): BatchNorm1d(57, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (1): Linear(in_features=57, out_features=512, bias=True)
  (2): ReLU()
  (3): Dropout(p=0.5, inplace=False)
  (4): Linear(in_features=512, out_features=256, bias=True)
  (5): ReLU()
  (6): Dropout(p=0.4, inplace=False)
  (7): Linear(in_features=256, out_features=128, bias=True)
  (8): ReLU()
  (9): Dropout(p=0.3, inplace=False)
  (10): Linear(in_features=128, out_features=16, bias=True)
  (11): ReLU()
  (12): Dropout(p=0.2, inplace=False)
  (13): Linear(in_features=16, out_features=10, bias=True)
)


In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=5e-4, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()
num_epochs = 300

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    # Training phase
    model.train()
    total_loss = 0

    for inputs, labels in tqdm(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Calculate the loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)

    # Testing phase
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_accuracy = correct / total
    print(f'Epoch: {epoch+1} / {num_epochs}, Train loss: {avg_train_loss:.4f}, Test accuracy: {test_accuracy:.4f}')

